# Procesamiento del Lenguaje Natural

## Generación de un modelo de lenguaje utilizando redes neuronales

Francisco Pablo Rodrigo

El modelo del lenguaje neuronal propuesto por Bengio (2003) es un modelo que estima probabilidades a partir de una red neuronal FeddForward. Como otros modelos, se puede entender como una tupla:

$$\mu = (\Sigma, P)$$

donde $\Sigma$ es el vocabulario de palabras y $P = p(w_j|w_i)$ es la probabilidad de transición de $w_i$ a $w_j$. En este caso $P$ es una red FeedForward con una arquitectura constituida por:

Una capa de embedding.
Una capa oculta con activación $\tanh$.
Una capa de salida con activación Softmax para obtener las probabilidades de transición.
A continuación mostramos una aplicación del modelo. No mostramos el modelo que en este caso es un script (LM_bengio). Por tanto, importamos este script y la paqueteria necesaria.

### Configuraciones previas

Se realizan los *imports* necesarios para la creación del modelo. Para el entrenamiento del modelo se puede hacer uso de PyTorch o Tensorflow o cualquier otra librería que tenga redes neuronales pre-entrenadas, sin embargo, para este ejercicio se creará la red desde cero utilizando simplemente _numpy_.

In [1]:
#-*- encoding:utf-8 -*-

import numpy as np
from collections import defaultdict, Counter
from itertools import chain

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize

from tqdm import tqdm
from sklearn.decomposition import PCA
from operator import itemgetter

Definimos tres funciones:

* Función para crear el vocabulario: asocia índices numéricos a palabras
* Función para asociar a cada elemento, una palabra
* Función para visualizar los embeddings por reducción de dimensionalidad con PCA

In [2]:
# Funcion que crea un vocabulario de palabras con un indice numero
def vocab():
    vocab = defaultdict()
    vocab.default_factory = lambda: len (vocab)
    return vocab

# Funcion que pasa la cadena de simbolos a una secuencia con indices numericos
def text2numba(corpus, vocab):
    for doc in corpus:
        yield [vocab[w] for w in doc]
        
#Función para visualizar los embeddings
#Usa reducción de la dimensionalidad por PCA
def plot_words(Z,ids):
    Z = PCA(2).fit_transform(Z)
    r=0
    plt.scatter(Z[:,0],Z[:,1], marker='o', c='blue')
    for label,x,y in zip(ids, Z[:,0], Z[:,1]):
        plt.annotate(label, xy=(x,y), xytext=(-1,1), textcoords='offset points', ha='center', va='bottom')
        r+=1
    plt.show()

### Elección de un corpus

Un **corpus** es una muestra bien organizada del nuestro lenguaje tomada de materiales escritos o hablados y que se encuentran agrupados bajo un críterio común.Para esta práctica se utilizará un corpus en *español*.

Obtenemos las sentencias con las que vamos a trabajar. Tokenizamos por oraciones y cada oración, a su vez, es tokenizada por palabras para obtener los elementos que servirán para el modelo del lenguaje.

Posteriormente, separamos los datos del corpus en el corpus de entrenamiento y el de evaluación:

In [3]:
# OPCION 1
sents =  [word_tokenize(s) for s in sent_tokenize(open('corpus/funes_el_memorioso.txt','r').read())]

#Split en corpus train y test
corpus, corpus_eval = train_test_split(sents, test_size=0.3)

print('Número de cadenas train:',len(corpus))
print('Número de cadenas test:',len(corpus_eval))

Número de cadenas train: 88
Número de cadenas test: 39


In [4]:
# OPCION 2
#import nltk
#nltk.download('gutenberg')
#sents = cess_esp.sents()
#nltk.download('punkt')
#sents = nltk.corpus.gutenberg.sents('carroll-alice.txt')

También podemos ver el número de tipos y tokens con el que cuenta el texto:

In [5]:
#Frecuencia de los tipos
freq_words= Counter( chain(*[' '.join(sent).lower().split() for sent in corpus]) )

print('Número de tipos: {} \nNúmero de tokens: {}'.format(len(freq_words), sum(freq_words.values())))

Número de tipos: 856 
Número de tokens: 2229


### Sustitución de los hapax

Ahora sustituiremos elementos del texto por el símbolo de fuera del vocabulario (Out Of Vocabulary) o $OOV$ esto nos permitirá manejar elementos que no se observen durante el entrenamiento.

In [6]:
#Nuevo corpus remplazando hápax por OOV
corpus_hapax = []
#Reemplazamos los hápax por OOV
for sent in corpus:
  sent_hapax =[]
  for w in sent:
    #Si es hápax
    if freq_words[w.lower()] == 1:
      #Se reemplaza por <oov>
      sent_hapax.append('<oov>')
    else:
      #De otra forma se mantiene la palabra en mínuscula
      sent_hapax.append(w.lower())
  #Se agrupan las cadenas    
  corpus_hapax.append(sent_hapax)
    
#print(corpus_hapax)

### 1.  Stemming

Para esta tarea no realiza el procesos de steamming con la finalidad de simplificar la validación del modelo, ya que de otra manera se deberían reconstruir las cadenas a la hora de evaluar el modelo o a la hora de usarlo para alguna aplicación, por ejemplo, la generación de oraciones.

### 2. Insertar símbolos de inicio y final de cadena

Se indexa cada simbolo del vocabulario previamente tratado (sin _stopwords_ y _estemmizado_) para tener un modelo de entrenamiento.

In [7]:
# Llamamos a la funcion para crear un vocabulario
idx = vocab() # Simplemente se renombra la funcion

cads_idx = list(text2numba(corpus_hapax,idx))

#print(idx)

Además, se colocarán etiquetas al inicio y al final de cada sentencia: BOS (Beginning of Sentence) y EOS (End of Sentence) respectivamente.

In [8]:
BOS = '<BOS>'
EOS = '<EOS>'

# A cada etiqueta se le asigna el indice número mayor 
# que el último indice asignado al vocabulario

BOS_IDX = max(idx.values()) + 2
EOS_IDX = max(idx.values()) + 1

# Se agregan las etiquetas al vocabulario
idx[EOS] = EOS_IDX
idx[BOS] = BOS_IDX

# Agregamos las etiquetas BOS al inicio y EOS al final de cada sentencia

strings = [[BOS_IDX] + cad + [EOS_IDX] for cad in cads_idx]

print(strings[:2])

[[194, 0, 1, 2, 1, 3, 4, 5, 6, 1, 3, 7, 8, 9, 10, 1, 1, 2, 11, 12, 13, 14, 1, 3, 1, 9, 7, 1, 3, 15, 16, 1, 2, 17, 18, 19, 20, 7, 1, 9, 21, 1, 22, 1, 1, 3, 15, 1, 11, 1, 4, 7, 1, 1, 9, 17, 1, 22, 1, 3, 1, 9, 11, 1, 22, 1, 3, 15, 1, 23, 1, 9, 1, 24, 1, 25, 1, 26, 27, 28, 29, 193], [194, 24, 1, 30, 1, 31, 1, 9, 15, 1, 8, 1, 29, 193]]


### 3. Bigramas

Antes de entrenar el modelo del lenguaje obtendremos los pares de entrenamiento que serán los pares obtenidos de bigramas, de tal forma que nuestro conjunto supervisado será:

$$\mathcal{S} = \{(i,j) : (w_i, w_j) \text{ es un bigrama}\}$$
Antes de obtener estos bigramas, además, debemos agregar los símbolos de $BOS$ y $EOS$, así como crear el vocabulario:

In [9]:
# Creacion de bigramas
bigrams = list(chain(*[zip(cad,cad[1:]) for cad in strings]))
print(len(bigrams))

2317


### 4. Entrenamiento de la red neuronal

In [ ]:
np.random.seed(0)

iterations = 50
eta = 0.1

# dim -> hiperparametro que define la dimensioón de los vectores-palabra
dim = 100
m = 300
N = len(idx)

# Se usa para generar vectores one-shot
matrix_I = np.identity(N)

# Embebidding
C = np.random.randn(dim,N) / np.sqrt(N)

# Oculta
W = np.random.randn(m,dim) / np.sqrt(dim)
b = np.ones(m)

# Salida
U = np.random.randn(N,m) / np.sqrt(m)
c = np.ones(N)


for i in tqdm(range(0,iterations)):    
    for bigram in bigrams:
        
        # FOWARD       
        
        # Capa embbeding
        c_i = C.T[bigram[0]]
        
        # Capa oculta
        h_i = np.tanh(np.dot(W,c_i) + b)
        
        # Pre-activacion
        a = np.dot(U,h_i) + c
        
        # Salidas
        tmp = np.exp(a - np.max(a))
        # Aplicando softmax
        f = tmp/tmp.sum(0)
        
        # BACKPROPAGATION para salida
        d_out = f
        k= bigram[1]
        d_out[k] -= 1
     
        # Backpropagation para la capa oculta
        dh = (1-h_i**2)*np.dot(U.T,d_out) 
        
        # Backpropagation para la capa embedding
        dc = np.dot(W.T,dh)
        c -= eta*d_out

        # Actualizacion de la capa de salida
        U -= eta*np.outer(d_out,h_i)
        
        # Actualizacion de capa oculta
        W -= eta*np.outer(dh,c_i)
        b -=eta*dh
        
        # Actualizacion embedding
        C -= eta*np.outer(dc,matrix_I[bigram[0]].T)

 60%|██████    | 30/50 [01:01<00:54,  2.70s/it]

### 5. Evaluación del modelo

Entrenada la red, definimos una función forward para obtener las probabilidades a partir de la red ya entrenada.

In [ ]:
def forward(x):
    # Capa embbeding
    c_i = C.T[x]
    # Capa oculta
    h_i = np.tanh(np.dot(W,c_i) + b)
    # Pre-activacion
    a = np.dot(U,h_i) + c
    # Salidas
    tmp = np.exp(a - np.max(a))
    # Aplicando softmax
    f = tmp/tmp.sum(0)
    
    return f

In [ ]:
lista = []

for word in idx.keys():
    lista.append((word,forward(idx['recuerdo'])[idx[word]]))
    #print(word,forward(idx['presidente'])[idx[word]])

lista.sort(key=lambda x: x[1],reverse=True)

lista[:10]

In [ ]:
label = [w[0] for w in sorted(idx.items(), key=itemgetter(1))]
plot_words(C.T[1:20],label[1:20])

Para evalaur el modelo, necesitamos primero definir una función que nos de la probabilidad de las cadenas. Definimos esta función a continuación

In [ ]:
def prob_sent(sent):
    #Obtenemos los simbolos
    seq = sent.split()
    #Obtenemos los bigramas de la cadena de evaluacion
    bigrSeq = zip(seq,seq[1:])
    
    #Guardamos la probabilidad inicial dado el modelo
    try:
        p = forward(idx['<BOS>'])[idx[seq[0]]]
    except: 
        p = forward(idx['<BOS>'])[idx['<oov>']]
    #Multiplicamos por las probabilidades de los bigramas dado el modelo
    for gram1, gram2 in bigrSeq:
        #Obtiene las probabilidades de transición
        #Dado el primer elemento
        try:
            prev_prob = forward(idx[gram1])
        #En caso de que sea una OOV
        except:
            prev_prob = forward(idx['<oov>'])
        #Obtiene la probabilidad de transitar a la siguiente palabra
        try:
            p *= prev_prob[idx[gram2]]
        #En caso de que sea una OOV
        except:
            p *= prev_prob[idx['<oov>']]
            
    return p

In [ ]:
prob_sent('los ojos cerrados')

Ya con esto, podemos evaluar el modelo con entropía empírica (tomamos el promedio por cadena de ésta). Asimismo, con base en la entropía empírica podemos obtener la perplejidad como: 
$$Px(\mu) = 2^{H(\mu)}$$

In [ ]:
#Evaluación del modelo
H = 0.0
for cad in corpus_eval:
    #Probabilidad de la cadena
    p_cad = prob_sent(' '.join(cad))
    #Longitud de la cadena
    M = len(cad)
    #Obtenemos la entropía cruzada de la cadena
    if p_cad == 0:
        pass
    else:
        H -= (1./M)*(np.log(p_cad)/np.log(2))
        
H = H/len(corpus_eval)

print('Entropía promedio: {}\nPerplejidad total: {}'.format(H,2**H))

In [ ]:
# 6. Calcular la proabilidad de 5 oraciones no vistas en el entrenamiento.
# 7. Guardar los vectores de la capa de embedding asociados a las palabras